# Python Data Analysis of Tech Gadgets Sales with Pandas

In [1]:
import pandas as pd
import os
import sys

## Merging Data Files

In [2]:
## Get the list of all files in the directory
file_path_data = "./data"
files = [file for file in os.listdir(file_path_data) if not file.startswith('.')] 
print(f"There are {len(files)} files in the directory: {file_path_data}.")
print(f"Here is the list of the files read: \n{files}")

df_all_months = pd.DataFrame()

## Combine data files
for file in files:
	## Current spreadsheet
	current_file = f"{file_path_data}/{file}"
	try: 
		df_month = pd.read_csv(current_file)
	except: 
		print(f"An error occurred while reading the file: {current_file}")
		sys.exit(1)
	## Concatenate Pandas DataFrame along the index axis 
	df_all_months = pd.concat([df_all_months, df_month])

## Output the combined data frames to 1 file
filename_output = "all-gadget-sales-2019.csv"
file_path_output = f"./output/{filename_output}"
df_all_months.to_csv(file_path_output, index=False)
# print(df_all_months.head())

## Read in the updated DataFrame
df_all_sales = pd.read_csv(file_path_output)
df_all_sales.reset_index()
# print(df_all_sales.head())

## Check the new DataFrame
# print(df_all_months.equals(df_all_sales))
# print(df_all_months.index.values)
# print(df_all_sales.index.values)

There are 12 files in the directory: ./data.
Here is the list of the files read: 
['gadget-sales-april-2019.csv', 'gadget-sales-august-2019.csv', 'gadget-sales-december-2019.csv', 'gadget-sales-february-2019.csv', 'gadget-sales-january-2019.csv', 'gadget-sales-july-2019.csv', 'gadget-sales-june-2019.csv', 'gadget-sales-march-2019.csv', 'gadget-sales-may-2019.csv', 'gadget-sales-november-2019.csv', 'gadget-sales-october-2019.csv', 'gadget-sales-september-2019.csv']


,index,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
...,...,...,...,...,...,...,...
186845,186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186846,186846,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186847,186847,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186848,186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


## Data Cleaning

The best way to find out what to clean is to perform operations and get errors. Based on the errors, we decide how to go about cleaning the data. 

In [3]:
## Find rows with all NA values
## isna() returns a boolean DataFrame that indicates whether each element is NA
## any(axis=1) reduces columns of the boolean DataFrame to only a boolean Series
df_nan = df_all_sales[df_all_sales.isna().any(axis=1)]
print(df_nan.head())

## Remove rows with all NA values
df_all_sales.dropna(how='all', inplace=True)
print(df_all_sales.head())

     Order ID Product Quantity Ordered Price Each Order Date Purchase Address
1         NaN     NaN              NaN        NaN        NaN              NaN
356       NaN     NaN              NaN        NaN        NaN              NaN
735       NaN     NaN              NaN        NaN        NaN              NaN
1433      NaN     NaN              NaN        NaN        NaN              NaN
1553      NaN     NaN              NaN        NaN        NaN              NaN
  Order ID                     Product Quantity Ordered Price Each  \
0   176558        USB-C Charging Cable                2      11.95   
2   176559  Bose SoundSport Headphones                1      99.99   
3   176560                Google Phone                1        600   
4   176560            Wired Headphones                1      11.99   
5   176561            Wired Headphones                1      11.99   

       Order Date                      Purchase Address  
0  04/19/19 08:46          917 1st St, Dallas, TX 750

When we created the `Month` column, we took the first 2 letters in the `Order Date` column and converted the `Month` column to the integer type, which resulted in `ValueError`. So, we came up here to perform some data cleaning operations to get rid of the text "Or" in `Order Date` column before we attempt the data conversion again.

In [4]:
# df_all_sales = df_all_sales[df_all_sales['Order Date'].str[0:2] != 'Or']
# df_all_sales.head()

#### Make columns correct type

In [5]:
df_all_sales['Quantity Ordered'] = pd.to_numeric(df_all_sales['Quantity Ordered'])
df_all_sales['Price Each'] = pd.to_numeric(df_all_sales['Price Each'])

ValueError: Unable to parse string "Quantity Ordered" at position 517

### Augment data with additional columns

#### Add month column

In [ ]:
df_all_sales['Month'] = df_all_sales['Order Date'].str[0:2]
df_all_sales['Month'] = df_all_sales['Month'].astype('int32')
df_all_sales.head()

#### Add month column (alternative method)

In [ ]:
df_all_sales['Month 2'] = pd.to_datetime(df_all_sales['Order Date']).dt.month
df_all_sales.head()

#### Add city column

In [ ]:
def get_city(address):
    return address.split(",")[1].strip(" ")

def get_state(address):
    return address.split(",")[2].split(" ")[1]

df_all_sales['City'] = df_all_sales['Purchase Address'].apply(lambda x: f"{get_city(x)}  ({get_state(x)})")
df_all_sales.head()

## Data Exploration!

#### Question 1: What was the best month for sales? How much was earned that month? 

In [ ]:
df_all_sales['Sales'] = df_all_sales['Quantity Ordered'].astype('int') * df_all_sales['Price Each'].astype('float')

In [ ]:
df_all_sales.groupby(['Month']).sum()

In [ ]:
import matplotlib.pyplot as plt

months = range(1,13)
print(months)

plt.bar(months,df_all_sales.groupby(['Month']).sum()['Sales'])
plt.xticks(months)
plt.ylabel('Sales in USD ($)')
plt.xlabel('Month number')
plt.show()



#### Question 2: What city sold the most product?

In [ ]:
df_all_sales.groupby(['City']).sum()

In [ ]:
import matplotlib.pyplot as plt

keys = [city for city, df in df_all_sales.groupby(['City'])]

plt.bar(keys,df_all_sales.groupby(['City']).sum()['Sales'])
plt.ylabel('Sales in USD ($)')
plt.xlabel('Month number')
plt.xticks(keys, rotation='vertical', size=8)
plt.show()

#### Question 3: What time should we display advertisements to maximize likelihood of customer's buying product?

In [ ]:
# Add hour column
df_all_sales['Hour'] = pd.to_datetime(df_all_sales['Order Date']).dt.hour
df_all_sales['Minute'] = pd.to_datetime(df_all_sales['Order Date']).dt.minute
df_all_sales['Count'] = 1
df_all_sales.head()


In [6]:
keys = [pair for pair, df in df_all_sales.groupby(['Hour'])]

plt.plot(keys, df_all_sales.groupby(['Hour']).count()['Count'])
plt.xticks(keys)
plt.grid()
plt.show()

# My recommendation is slightly before 11am or 7pm

KeyError: 'Hour'

#### Question 4: What products are most often sold together?

In [ ]:
# https://stackoverflow.com/questions/43348194/pandas-select-rows-if-id-appear-several-time
df = df_all_sales[df_all_sales['Order ID'].duplicated(keep=False)]

# Referenced: https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
df2 = df[['Order ID', 'Grouped']].drop_duplicates()

In [ ]:
# Referenced: https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary
from itertools import combinations
from collections import Counter

count = Counter()

for row in df2['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key,value in count.most_common(10):
    print(key, value)


#### What product sold the most? Why do you think it sold the most?

In [7]:
product_group = df_all_sales.groupby('Product')
quantity_ordered = product_group.sum()['Quantity Ordered']

keys = [pair for pair, df in product_group]
plt.bar(keys, quantity_ordered)
plt.xticks(keys, rotation='vertical', size=8)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Referenced: https://stackoverflow.com/questions/14762181/adding-a-y-axis-label-to-secondary-y-axis-in-matplotlib

prices = df_all_sales.groupby('Product').mean()['Price Each']

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(keys, quantity_ordered, color='g')
ax2.plot(keys, prices, color='b')

ax1.set_xlabel('Product Name')
ax1.set_ylabel('Quantity Ordered', color='g')
ax2.set_ylabel('Price ($)', color='b')
ax1.set_xticklabels(keys, rotation='vertical', size=8)

fig.show()